# Method 3(b): SVM using DWave Quantum Annealer



In [ ]:
# install kaggle
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# download data set from Kaggle
! kaggle datasets download pcbreviglieri/pneumonia-xray-images

 99% 1.13G/1.14G [00:12<00:00, 114MB/s] 
100% 1.14G/1.14G [00:12<00:00, 99.2MB/s]


In [ ]:
! unzip pneumonia-xray-images

from IPython.display import clear_output
clear_output()

In [ ]:
# import required libraries
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
from matplotlib import image
from matplotlib import pyplot
from os import listdir
from numpy import save
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array

In [ ]:
# resize opacity images from training set

folder = '/content/train/opacity'
photos_op1 = list()

n1 = 200
n2 = 200

# enumerate files in the directory
for file in listdir(folder):
    # load image
    photo = load_img( '/content/train/opacity/'+ file, target_size=(n1, n2), color_mode='grayscale')
    # convert to numpy array
    photo = img_to_array(photo)
    # store in the list of photos
    photos_op1.append(photo)

# convert list of photos and labels to numpy arrays
photos_op = np.array(photos_op1[:1000])

# label = -1 for opacity cases
labels_op = np.array([-1.0]*1000)

op_train = list()
for i in range(1000):
  op_train.append(photos_op[i].reshape(-1))
op_train = np.array(op_train)
op_train = op_train/255.0

print(op_train.shape)

(1000, 40000)


In [ ]:
# resize normal images from training set
folder = '/content/train/normal'
photos_no = list()

# enumerate files in the directory
for file in listdir(folder):
    # load image
    photo = load_img( '/content/train/normal/'+ file, target_size=(n1, n2), color_mode='grayscale')
    # convert to numpy array
    photo = img_to_array(photo)
    # store in the list of photos
    photos_no.append(photo)

# convert list of photos and labels to numpy arrays
photos_no = np.array(photos_no[:1000])

# label = +1 for normal cases
labels_no = np.array([1.0]*1000)

no_train = list()
for i in range(1000):
  no_train.append(photos_no[i].reshape(-1))
no_train = np.array(no_train)
no_train = no_train/255.0

print(no_train.shape)

(1000, 40000)


In [ ]:
# for normal case images in test data set
folder = '/content/val/normal'
photos_test_no = list()

# enumerate files in the directory
for file in listdir(folder):
    # load image
    photo = load_img( '/content/val/normal/'+ file, target_size=(n1, n2), color_mode='grayscale')
    # convert to numpy array
    photo = img_to_array(photo)
    # store in the list of photos
    photos_test_no.append(photo)

# convert list of photos and labels to numpy arrays
photos_test_no = np.array(photos_test_no)

# label = +1 for normal cases
labels_test_no = np.array([1.0]*photos_test_no.shape[0])

no_test = list()
for i in range(267):
  no_test.append(photos_test_no[i].reshape(-1))
no_test = np.array(no_test)
no_test = no_test/255.0

print(no_test.shape)

(267, 40000)


In [ ]:
# for opacity case test data take extra images from train folder that are not used for training

# convert list of photos and labels to numpy arrays
photos_test_op = np.array(photos_op1[1000:2000])

# label = -1 for normal cases
labels_test_op = np.array([-1.0]*photos_test_op.shape[0])

op_test = list()
for i in range(1000):
  op_test.append(photos_test_op[i].reshape(-1))
op_test = np.array(op_test)
op_test = op_test/255.0

print(op_test.shape)

(1000, 40000)


In [ ]:
# divide all images and labels in sets of 50 each
train = np.empty((40, 50, 40000))
lab = np.empty((40, 50))
for i in range(40):
    train[i] = np.append(op_train[25*i:25*(i+1)], no_train[25*i:25*(i+1)], axis = 0)
    lab[i] = np.append(labels_op[25*i:25*(i+1)], labels_no[25*i:25*(i+1)])

print(train.shape, lab.shape)

(40, 50, 40000) (40, 50)


In [ ]:
# define RBF kernel
import math
def Kernel(mat1, mat2, sigma):
  norm=np.linalg.norm(mat1-mat2)
  k = -(0.5*norm**2)/(sigma**2)
  return np.exp(k)

In [ ]:
# evaluate kernel matrix for all image sets
x=list() # x is the list of all 40 kernel matrices
for k in range(40):
  a = np.empty((50,50))
  for i in range(50):
    for j in range(50):
      a[i][j]=Kernel(train[k][i],train[k][j],50)
  x.append(a)

In [ ]:
# Computing the QUBO matrix for each SVM

I=np.identity(50)
p=[1,2]
pp=np.kron(I,p)

Q=list() # Q is the list of 40 QUBO matrices
for k in range(40):
 y = np.empty((50,50))
 for i in range(50):
   for j in range(50):
     y[i][j] = lab[k][i]*lab[k][j]
 xy=np.multiply(x[k],y)
 q=np.dot(pp.T,np.dot(xy,pp))*0.5
 q=q-np.diag(np.dot(pp.T,np.ones(50)))
 Q.append(q)

In [ ]:
!pip install dwave-ocean-sdk

In [ ]:
# Import the Dwave packages dimod and neal
import dimod
import neal
# Import Matplotlib to generate plots
import matplotlib.pyplot as plt
# Import numpy and scipy for certain numerical calculations below
from scipy.special import gamma
import math
from collections import Counter
import pandas as pd
from itertools import chain
import time
import networkx as nx

In [ ]:
#setup dwave using api token
!dwave setup
!dwave ping

In [ ]:
import dwave_networkx as dnx
from dwave.system import (DWaveSampler, EmbeddingComposite,
                          FixedEmbeddingComposite)
from pprint import pprint

In [ ]:
# perform annealing for 40 SVMs
X=list()
for i in range(40):
 model = dimod.BinaryQuadraticModel.from_qubo(Q[i])
 DWavesampler = EmbeddingComposite(DWaveSampler(solver='Advantage_system6.2'))
 DWaveSamples = DWavesampler.sample(bqm=model, num_reads=500,
                                   return_embedding=True
                                   )
 energies = DWaveSamples.data_vectors['energy']
 mit = min(energies)
 print(mit)
 # save the values of lagrange multipliers that give minimum energy
 for sample,energy, num_occ in DWaveSamples.data(['sample','energy','num_occurrences']):
  if (energy==mit):
       X.append(np.array(list(sample.values())))
       break

-19.675827934516107
-33.981442202561574
-19.752734752302388
-21.065911149538596
-19.062788332283063
-27.96645771534544
-30.960153597537243
-45.8520778548065
-24.610671709311806
-12.010460700404165
-36.35772815903848
-25.269497604147027
-23.982537106715583
-31.79445617326104
-28.898921031214496
-24.39099139564694
-28.234232724453015
-22.91973451300911
-22.076027646214555
-32.66995950996122
-37.4940805858032
-34.76541167891242
-4.854534228634453
-45.81842783217654
-28.33092995118283
-28.65673820964405
-29.28289521103744
-33.0241941537798
-26.830834750905968
-20.384645310356788
-18.684637859122773
-32.71185638273567
-21.195767838043093
-33.35246548904198
-36.42247257300272
-27.949573677000018
-28.127906009437847
-33.04069921076666
-38.95860520486609
-34.23933963474774


In [ ]:
# convert values from binary to integers
xa=list()
for i in range(40):
  xa.append(np.matmul(pp,X[i].T))

In [53]:
# calculate bias for each SVM and store values in a list
bias=list()
for i in range(40):
 b=0
 c=0
 for n in range(50):
   a=0
   for m in range(50):
    a=a+xa[i][m]*lab[i][m]*x[i][m][n]
   b=b+xa[i][n]*(3-xa[i][n])*(lab[i][n]-a)
   c=c+xa[i][n]*(3-xa[i][n])
 b = b/c
 bias.append(b)

In [ ]:
# testing the model on normal images from test data
yy=list()
for i in range(267):
  a=0.00
  tes=list()
  for j in range(40):
   for k in range(50):
    a=a+xa[j][k]*lab[j][k]*Kernel(no_test[i],train[j][k],50)
   a=a+bias[j]
   if a<0:
    tes.append(-1)
   else:
    tes.append(1)
  yy.append(max(set(tes), key=tes.count))

In [ ]:
count = 0
for i in range(267):
  if yy[i]==1:
    count = count + 1
print(count)

221


In [ ]:
# testing the model on opacity images from test data
yy=list()
for i in range(1000):
  a=0.00
  tes=list()
  for j in range(40):
   for k in range(50):
    a=a+xa[j][k]*lab[j][k]*Kernel(op_test[i],train[j][k],50)
   a=a+bias[j]
   if a<0:
    tes.append(-1)
   else:
    tes.append(1)
  yy.append(max(set(tes), key=tes.count))

In [ ]:
count = 0
for i in range(1000):
  if yy[i]==-1:
    count = count + 1
print(count)

924


Thus, the model has labelled 924 out of 1000 opacity images and 221 out of 267 normal images correctly. The results can be expressed in a confusion matrix as follows:

|  | Actual +ve | Actual -ve |
| --- | --- | --- |
| **Predicted +ve** | 924 | 46 |
| **Predicted -ve** | 76 | 221 |

Here, positive is opacity and negative is normal.